#### https://chatgpt.com/share/c9474818-b9fe-4e2b-ad4b-fe037d073240

#### Here is a Python script to train and evaluate a single-layer neural network on the MNIST dataset using Keras:

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape((x_train.shape[0], 28 * 28)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], 28 * 28)).astype('float32') / 255

# Convert labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the model
model = models.Sequential()
model.add(layers.Dense(10, activation='softmax', input_shape=(28 * 28,)))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')


2024-09-02 07:19:34.276529: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2024-09-02 07:19:34.308209: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2024-09-02 07:19:34.309504: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4e7dbb0 executing computations on platform Host. Devices:
2024-09-02 07:19:34.309526: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version


Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 2s 32us/sample - loss: 0.7410 - accuracy: 0.8175 - val_loss: 0.4060 - val_accuracy: 0.8957
Epoch 2/10
48000/48000 [==============================] - 1s 20us/sample - loss: 0.3887 - accuracy: 0.8965 - val_loss: 0.3335 - val_accuracy: 0.9105
Epoch 3/10
48000/48000 [==============================] - 1s 19us/sample - loss: 0.3380 - accuracy: 0.9083 - val_loss: 0.3109 - val_accuracy: 0.9144
Epoch 4/10
48000/48000 [==============================] - 1s 21us/sample - loss: 0.3142 - accuracy: 0.9136 - val_loss: 0.2931 - val_accuracy: 0.9176
Epoch 5/10
48000/48000 [==============================] - 1s 21us/sample - loss: 0.3000 - accuracy: 0.9172 - val_loss: 0.2840 - val_accuracy: 0.9207
Epoch 6/10
48000/48000 [==============================] - 1s 19us/sample - loss: 0.2902 - accuracy: 0.9197 - val_loss: 0.2799 - val_accuracy: 0.9212
Epoch 7/10
48000/48000 [==============================] 

Test accuracy: 0.9250


####  Automate the process of generating the C code with the trained weights and biases using a Python script. 

In [2]:
import numpy as np

# Assuming the weights and biases from the trained Keras model
weights = model.layers[0].get_weights()[0]  # Extract weights
biases = model.layers[0].get_weights()[1]   # Extract biases

# Flatten the weights matrix in the correct order
weights_flattened = weights.flatten()

# Generate C code for weights and biases
def generate_c_code(weights, biases):
    c_code = f"""
#include <math.h>

#define INPUT_SIZE {weights.shape[0]}
#define OUTPUT_SIZE {weights.shape[1]}

float weights[OUTPUT_SIZE][INPUT_SIZE] = {{
"""
    for i in range(weights.shape[1]):
        c_code += "    {"
        c_code += ", ".join([f"{weights[j, i]:.6f}" for j in range(weights.shape[0])])
        c_code += "},\n"
    c_code += "};\n\n"

    c_code += "float biases[OUTPUT_SIZE] = {"
    c_code += ", ".join([f"{bias:.6f}" for bias in biases])
    c_code += "};\n\n"

    c_code += """
void predict(float input[INPUT_SIZE], float output[OUTPUT_SIZE]) {
    for (int i = 0; i < OUTPUT_SIZE; i++) {
        output[i] = biases[i];
        for (int j = 0; j < INPUT_SIZE; j++) {
            output[i] += input[j] * weights[i][j];
        }
    }
}
"""
    return c_code

# Generate the C code
c_code = generate_c_code(weights, biases)

# Save the generated C code to a file
with open("mnist_inference.c", "w") as file:
    file.write(c_code)

print("C code has been generated and saved to 'mnist_inference.c'")


C code has been generated and saved to 'mnist_inference.c'


#### To validate the accuracy of the generated mnist_inference.c against the MNIST test dataset, we write a Python script that generates another C code. This C code will load the MNIST test dataset, feed each image into the mnist_inference.c functions, and calculate the accuracy.

In [3]:
import numpy as np
import struct
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the MNIST test dataset
(_, _), (x_test, y_test) = mnist.load_data()

# Flatten and normalize the test images
x_test = x_test.reshape((x_test.shape[0], 28 * 28)).astype('float32') / 255.0

def generate_test_c_code(x_test, y_test):
    c_code = """
#include <stdio.h>
#include <math.h>
#include "mnist_inference.c"

#define NUM_TESTS 10000

float test_inputs[NUM_TESTS][INPUT_SIZE] = {
"""
    # Add test inputs to the C code
    for i in range(x_test.shape[0]):
        c_code += "    {"
        c_code += ", ".join([f"{x_test[i, j]:.6f}" for j in range(x_test.shape[1])])
        c_code += "},\n"
    c_code += "};\n\n"

    # Add test labels to the C code
    c_code += "int test_labels[NUM_TESTS] = {\n    "
    c_code += ", ".join([str(label) for label in y_test])
    c_code += "\n};\n\n"

    # Add accuracy calculation code
    c_code += """
int main() {
    int correct_predictions = 0;
    float output[OUTPUT_SIZE];

    for (int i = 0; i < NUM_TESTS; i++) {
        // Perform prediction
        predict(test_inputs[i], output);

        // Find the index of the maximum output (predicted class)
        int predicted_label = 0;
        float max_value = output[0];
        for (int j = 1; j < OUTPUT_SIZE; j++) {
            if (output[j] > max_value) {
                max_value = output[j];
                predicted_label = j;
            }
        }

        // Check if the prediction is correct
        if (predicted_label == test_labels[i]) {
            correct_predictions++;
        }
    }

    // Calculate and print accuracy
    float accuracy = (float)correct_predictions / NUM_TESTS * 100.0;
    printf("Accuracy: %f%%\\n", accuracy);

    return 0;
}
"""
    return c_code

# Generate the test C code
test_c_code = generate_test_c_code(x_test, y_test)

# Save the generated test C code to a file
with open("mnist_test.c", "w") as file:
    file.write(test_c_code)

print("Test C code has been generated and saved to 'mnist_test.c'")


Test C code has been generated and saved to 'mnist_test.c'


#### Write Python Code to Generate params.c: This Python script will generate the params.c file with hardcoded weights and biases.

In [4]:
import numpy as np

# Replace this with your actual weights and biases
weights = np.random.randn(10, 784).tolist()  # Example weights
biases = np.random.randn(10).tolist()        # Example biases

# Write weights to params.c
with open('params.c', 'w') as file:
    file.write('#include <stdint.h>\n\n')
    
    # Write weights
    file.write('const float weights[10][784] = {\n')
    for i in range(10):
        file.write('    {')
        file.write(', '.join(f'{w:.6f}' for w in weights[i]))
        file.write('},\n')
    file.write('};\n\n')
    
    # Write biases
    file.write('const float biases[10] = {\n')
    file.write(', '.join(f'{b:.6f}' for b in biases))
    file.write('};\n')

print("Weights/bias has been saved to 'params.c'")

Weights/bias has been saved to 'params.c'


####  Compile and Run:

In [6]:
!gcc mnist_test.c -o mnist_test -lm

In [7]:
!./mnist_test 

Accuracy: 92.500000%


#### Using LLM, convert the `mnist_inference.c` to HLS C.